In [1]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import time
import random as rd

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

2023-01-09 14:39:59.804690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### **Data importation & visualization**

In [2]:
metadata = pd.read_csv("ISIC_2020_Training_GroundTruth_v2.csv")
metadata

,image_name,patient_id,lesion_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0
...,...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,IL_2076932,male,50.0,torso,unknown,benign,0
33122,ISIC_9999320,IP_3650745,IL_6891604,male,65.0,torso,unknown,benign,0
33123,ISIC_9999515,IP_2026598,IL_6364820,male,20.0,lower extremity,unknown,benign,0
33124,ISIC_9999666,IP_7702038,IL_6048457,male,50.0,lower extremity,unknown,benign,0


In [3]:
data_dir = "data/jpg/"
alexNet_dir = "data/227x227/"

data_paths = os.listdir(data_dir)
alexNet_paths = os.listdir(alexNet_dir)

In [4]:
df = metadata[["image_name", "benign_malignant"]]
df["y"] = df["benign_malignant"] == "malignant"
df["y"] = df["y"] * 1
df

/tmp/ipykernel_1493211/47819270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"] = df["benign_malignant"] == "malignant"
/tmp/ipykernel_1493211/47819270.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"] = df["y"] * 1


,image_name,benign_malignant,y
0,ISIC_2637011,benign,0
1,ISIC_0015719,benign,0
2,ISIC_0052212,benign,0
3,ISIC_0068279,benign,0
4,ISIC_0074268,benign,0
...,...,...,...
33121,ISIC_9999134,benign,0
33122,ISIC_9999320,benign,0
33123,ISIC_9999515,benign,0
33124,ISIC_9999666,benign,0


In [5]:
def reshapeImages(dim):
  if not os.path.exists(f"data/{dim}x{dim}/"):
    os.makedirs(f"data/{dim}x{dim}/")
  for img_path in os.listdir(data_dir):
    img = Image.open(data_dir + img_path)
    new_img = img.resize((dim, dim))
    new_img.save(f"data/{dim}x{dim}/" + img_path)

In [6]:
def getY(image_name):
  image_name = image_name.strip(".jpg")
  img = df.loc[df['image_name'] == str(image_name)]
  res = img["y"]
  return res.values[0]

In [7]:
# reshapeImages(227)

In [8]:
df

,image_name,benign_malignant,y
0,ISIC_2637011,benign,0
1,ISIC_0015719,benign,0
2,ISIC_0052212,benign,0
3,ISIC_0068279,benign,0
4,ISIC_0074268,benign,0
...,...,...,...
33121,ISIC_9999134,benign,0
33122,ISIC_9999320,benign,0
33123,ISIC_9999515,benign,0
33124,ISIC_9999666,benign,0


In [9]:
def splitData(train_percentage, validation_percentage, test_percentage):

  if train_percentage + validation_percentage + test_percentage != 1:
    return False

  img_paths = os.listdir(alexNet_dir)[:10000] # limit the db

  total_data_len = len(img_paths)

  train_len = np.round(total_data_len * train_percentage).astype("int")
  validation_len = np.round(total_data_len * validation_percentage).astype("int")
  test_len = np.round(total_data_len * test_percentage).astype("int")

  rd.shuffle(img_paths)

  train_paths = img_paths[:train_len]
  validation_paths = img_paths[train_len:train_len+validation_len]
  test_paths = img_paths[train_len+validation_len:]

  return train_paths, validation_paths, test_paths

# tr_p, val_p, test_p = splitData(0.8, 0.1, 0.1)
train_paths, validation_paths, test_paths = splitData(0.8, 0.1, 0.1)

In [10]:
def getLabels(paths):
  labels = []
  for im_path in paths:
    labels.append(getY(im_path))
  return np.resize(np.array(labels), (len(labels), 1))

# print(getLabels(val_p))

In [11]:
def pathsToArray(paths):
  data_arr = []
  for img_path in paths:
    path = alexNet_dir + img_path
    im_arr = np.array(Image.open(path))
    data_arr.append(im_arr)
  return data_arr

In [12]:
def bigPathToArray(paths):
  arr = []
  step = 2640
  for i in range(10):
    print("From: ", i*step)
    print("To: ", (i+1)*step)
    interm_path = paths[i*step:(i+1)*step]

    for img_path in interm_path:
      path = alexNet_dir + img_path
      im_arr = np.array(Image.open(path))
      # arr = arr + im_arr
      arr.append(im_arr)

  return arr


# train_images = bigPathToArray(train_paths)

In [13]:
def load_data(train_portion, val_portion, test_portion):
  train_paths, validation_paths, test_paths = splitData(train_portion, val_portion, test_portion)
  
  train_images = pathsToArray(train_paths)
  train_labels = getLabels(train_paths)

  validation_images = pathsToArray(validation_paths)
  validation_labels = getLabels(validation_paths)

  test_images = pathsToArray(test_paths)
  test_labels = getLabels(test_paths)

  return (train_images, train_labels), (validation_images, validation_labels), (test_images, test_labels)

In [14]:
train_images = pathsToArray(train_paths)
train_labels = getLabels(train_paths)

validation_images = pathsToArray(validation_paths)
validation_labels = getLabels(validation_paths)

test_images = pathsToArray(test_paths)
test_labels = getLabels(test_paths)

#### **AlexNet Model**

In [ ]:
# (train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()
(train_images, train_labels), (validation_images, validation_labels), (test_images, test_labels) = load_data(0.8, 0.1, 0.1)

In [15]:
print(len(train_images))
print(len(train_images[0]))
print(len(train_images[0][0]))
print(len(train_images[0][0][0]))
# print(train_labels)

8000
227
227
3


In [16]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

: 

: 

In [ ]:
CLASS_NAMES= ["benign", "malignant"]

In [ ]:
plt.figure(figsize=(20,20))
for i, (image, label) in enumerate(train_ds.take(5)):
    ax = plt.subplot(5,5,i+1)
    plt.imshow(image)
    plt.title(CLASS_NAMES[label.numpy()[0]])
    plt.axis('off')

In [ ]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    return image, label

In [ ]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()

print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

In [ ]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
                  
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    keras.layers.Flatten(),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    keras.layers.Dense(2, activation='sigmoid')
])

In [ ]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_ds,
          epochs=50,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])